<a href="https://colab.research.google.com/github/njanrepo/MLEProject/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import math

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 18.9 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import yfinance as yf

### Task1 :: Get companies

In [4]:
def get_nyse_ticker_list():
  others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
  ticker_df = pd.read_csv(others_list, sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

In [5]:
def get_sp500_list():
  payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  first_table = payload[0]
  sp500df = first_table
  sp500_list = sp500df.Symbol.values.tolist()
  return sp500_list

In [6]:
def get_stock_data(type='S&P500', limit=100):
  ticker_list = get_nyse_ticker_list() if type == 'NYSE' else get_sp500_list()
  if(limit > len(ticker_list)):
    limit = len(ticker_list)
  ticker_data  = yf.download(ticker_list[0:limit], period='6mo', progress=True, group_by='symbol')
  try:
    ticker_data.drop('BF.B', axis=1, inplace=True)
    ticker_data.drop('BRK.B', axis=1, inplace=True)
  except:
    print("its fine!!")
  ticks = ticker_data.columns.get_level_values(0).unique().tolist()
  return (ticker_data, ticks)




#Calculate Market Capitalization


In [7]:
#wrong, it is just volum, not total outstanding shares
ticker='JPM'
def calc_market_capitalization(symbol):
  symbol_data = yf.download(symbol, period='1d')
  print(symbol_data)
  return round((symbol_data['Volume'][0] * symbol_data['Close'][0]),2)
print(calc_market_capitalization(ticker))

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close   Volume
Date                                                                           
2021-11-16  167.059998  167.070007  165.119995  165.360001  165.360001  9706145
1605008143.12


In [8]:
def ask_user(stocks = None):
  if(stocks != None):
    return stocks;
  how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  correct = False
  while not correct:
    try:
      how_many_stocks = int(how_many_stocks)
      if(how_many_stocks > 500 or how_many_stocks < 10):
        raise ValueError
      correct = True
    except:
      print("Please enter correct number 10 to 500")
      how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  return how_many_stocks    

In [9]:
def make_X_and_Y(close_prices, period):
  X = []
  y = []
  for i in range(len(close_prices)- period):
    X.append(close_prices[i:i+period])
    y.append(close_prices[i+period])
  return X,y

In [10]:
def cl_make_X_and_Y(close_prices, period, in_future):
  X = []
  y = []
  for i in range(len(close_prices)- period-in_future):
    X.append(close_prices[i:i+period])
    l = close_prices[i+period:i+period+in_future]
    y.append(calculate_label(l))
  return X,y

def calculate_label(params):
  changes = []
  for j in range(len(params)-1):
    changes.append(params[j+1] - params[j])
  tot = sum(changes)
  if tot > 0:
    return 1;
  elif tot < 0:
     return -1;
  else:
     return 0;


In [11]:
dt, tick = get_stock_data(limit=50)

x,y=cl_make_X_and_Y(dt['ABBV'].Close, 6, 5)


[*********************100%***********************]  50 of 50 completed
its fine!!


In [12]:
def get_user_selected_stocks():
  u = input("Enter ticker of stocks seperated by space..")
  return u.split(' ');

def get_user_selected_weightage(no_stocks):
  w = input("Enter weightage per stocks seperated by space..\n Eg: 80 20 \n")
  resp = False
  while not resp:
    try:
      proportion = w.split(' ')
      if no_stocks != len(proportion):
        print('Incorrect weightage')
        raise ValueError
      if  sum(map(int, proportion)) < 0 or sum(map(int, proportion)) > 100 :
        print('sum of weightages are incorrect :', proportion)
        raise ValueError
      resp = True
    except:
      w = input("Enter weightage per stocks seperated by space..\n Eg: 80 20 \n")
  return w.split(' ');

def get_user_strategy():
  print('\nSelect your strategy: ') 
  s= get_correct_input_from_user("\t1. Market Capital Weighted\n\t2. Equal Weighted\n\t3. User defined proportion\n")
  while not (s > 0 and s <=3):
    print("Enter correct option 1-3")
    s=get_correct_input_from_user("\t1. Market Capital Weighted\n\t2. Equal Weighted\n\t3. User defined proportion\n")
  return s

def user_selected_analysis(income):
  selected = get_user_selected_stocks()
  strategy = get_user_strategy()
  market_cap = []
  close_price = []
  for sym in selected:
    try:
      current_data = yf.download(sym, period='1d', progress=False)
      close_price.append(current_data.Close[0])
      market_cap.append(current_data.Close[0] * current_data.Volume[0])
    except:
      selected.remove(sym)
      print('Data not found for symbol :: ', sym)
  s = sum(market_cap)
  price = []
  if strategy == 1:
    print ('Calculating Market Cap Weighted portfolio')
  elif strategy == 2:
    print('Calculating Equal Weighted portfolio') 
  else:
    proportion = get_user_selected_weightage(len(selected))
    print('Calculating user defined Weighted portfolio :: ', proportion)
    
  port_list = []
  for i, cap in enumerate(market_cap):
    d = {}
    if strategy == 3 and proportion != None:
      weight = int(proportion[i])
      portion = (weight/100)*income
    elif strategy == 1:
      portion = (income* cap)/s
    else:
      weight = (1/len(selected))
      portion = weight*income
    
    no_stocks = math.floor(portion/close_price[i])
    price.append(round(no_stocks * close_price[i],2))
    d['ticker'] = selected[i]
    d['stocks_to_purchase'] = no_stocks
    d['price'] = round(close_price[i],2)
    d['market_val'] = round(close_price[i] * no_stocks, 2)
    d['planned_portion'] = str(round((portion/income) * 100, 2)) +'%'
    d['portfoloio_portion'] = str(round((d['market_val']/income) * 100, 2)) +'%'
    port_list.append(d)
  c={}
  c['ticker'] = 'CASH'
  c['stocks_to_purchase'] = income - sum(price)
  c['price'] = 1
  c['market_val'] = income - sum(price)
  c['planned_portion'] = '-'
  c['portfoloio_portion'] = str(round((c['market_val']/income) * 100, 2)) +'%'
  port_list.append(c)
  #return pd.DataFrame({'ticker': selected, 'stocks_to_purchase': stocks_to_purchase, 'price': price})
  return pd.DataFrame(port_list)

#w = user_selected_analysis(10000)
#gft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN'], 10000)
#eft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN' ], 10000, 'equal')
#uft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN' ], 10000, proportion=w)
#print(gft[['market_val', 'planned_portion', 'portfoloio_portion']])
#print(eft[['market_val', 'planned_portion', 'portfoloio_portion']])
#print(uft[['market_val', 'planned_portion', 'portfoloio_portion']])

In [13]:
def get_correct_input_from_user(text):
  i = input(text)
  resp = False
  while not resp:
    try:
      a= int(i)
      resp = True
    except ValueError:
      print("Unable to parse number. Enter correct number..")
      i = input(text)
  return a


In [14]:
income = get_correct_input_from_user("How much money do you want to invest?  ")
print('\nDo you wish to add you own stocks or want ML to recommend you? ')
user_port_option = get_correct_input_from_user("\tEnter 1 for adding you own stock\n\tEnter 2 for ML recommendations.  ")
if user_port_option == 1:
  x=user_selected_analysis(income)
  print(x[['ticker', 'stocks_to_purchase', 'planned_portion', 'price']])
elif user_port_option ==2:
  
  print('sdf')
else:
  print("Enter only 1 or 2, No other numbers are accepted.")

How much money do you want to invest?  1

Do you wish to add you own stocks or want ML to recommend you? 
	Enter 1 for adding you own stock
	Enter 2 for ML recommendations.  1
Enter ticker of stocks seperated by space..AAPL AMZA

Select your strategy: 
	1. Market Capital Weighted
	2. Equal Weighted
	3. User defined proportion
1
Calculating Market Cap Weighted portfolio
  ticker  stocks_to_purchase planned_portion   price
0   AAPL                 0.0          99.98%  151.00
1   AMZA                 0.0           0.02%   28.12
2   CASH                 1.0               -    1.00


In [15]:
def return_model(X, y):
  clf = RandomForestRegressor(n_estimators = 200)
  clf.fit(X,y)
  return clf

In [25]:

def predict_price(symbol, period, data=None):
  print("\rAnalysing :: ", symbol, end="" )
  if(data is None):
    data = yf.download(symbol, period='6mo', progress=False, )
  price = data.Close
  features, dep_var = make_X_and_Y(list(price), period)
  features = np.array(features)
  dep_var = np.array(dep_var)
  clf=return_model(features, dep_var) 
  
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  s_dict = {}
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = round(clf.predict(latest_feature)[0], 2)
  s_dict['today'] = round(price[-1],2)
  return s_dict

In [17]:
predict_price('MMM', 4)

Analysing ::  MMM


{'symbol': 'MMM', 'today': 183.37, 'tomorrow': 182.78, 'volatility': 9.46}

In [28]:
def prepare_recomm(data, how_many_stocks = None, period = None):
  #list_of_symbols = get_sp500_list()
  if(data is None):
    print("No stock data found..")
    return
  list_of_symbols = data.columns.get_level_values(0).unique().tolist()
  if period == None:
    period = 6
  stk_data = []
  if how_many_stocks == None:
    how_many_stocks = ask_user()
  if how_many_stocks > len(list_of_symbols):
    print("No. of stocks set to max")  
    how_many_stocks = len(data)
  for sym in list_of_symbols[0:how_many_stocks]:
    stk_data.append(predict_price(sym, period,data[sym]))
  return pd.DataFrame(stk_data)


In [19]:
def get_recomm(s_data, ascending):
  cat = 'conservative' if ascending else 'aggressive'
  print("Top 5 " + cat +" stocks are.. ")
  tmp = s_data.sort_values(by='volatility', ascending=ascending)
  tmp = tmp[tmp['tomorrow'] > tmp['today']]
  return tmp.head()  

In [20]:
#get_stock_data takes two param, 
  #type: S&P500(default) or 'NYSE, 
  #limit: default 100
data_df, ticker_list = get_stock_data()

[*********************100%***********************]  100 of 100 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [29]:
pred_df = prepare_recomm(data_df, 500, 6)

Analysing ::  BMY


In [23]:
has_given_resp = False
strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")
while not has_given_resp:
  if strategy == '1' or strategy.lower() == 'c' or strategy.lower() == 'conservative':
    has_given_resp = True
    print(get_recomm(pred_df, True))
  elif strategy == '2' or strategy.lower() == 'a' or strategy.lower() == 'aggresive':
    has_given_resp = True
    print(get_recomm(pred_df, False))
  else:
    print("Please enter correct number")
    strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")


Enter Strategy : 
 Enter 1 for Conservative
 Enter 2 for Aggresive
2
Top 5 aggressive stocks are.. 
   symbol  volatility  tomorrow    today
2   GOOGL      201.56   2962.45  2957.52
71   BKNG      125.21   2414.22  2380.33
8    ALGN       48.87    692.22   689.06
63   BIIB       39.86    267.05   261.55
89   TECH       35.28    505.32   504.49
